# **Adecuacion de la plataforma**

In [0]:
import tensorflow as tf

In [0]:
# instala el ambiente de spark..solo se corre una vez
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
#Configura variables de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
#importa package
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [0]:
# definir la sesion. 
conf = SparkConf().setAppName("ejemplo").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
# monta el google drive para usar sus archivos
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
#  INST A: Opcional..si quiere subir archivos al ambiente de trabajo
from google.colab import files
datafile = files.upload()

Saving datos.txt to datos (1).txt


# **Example 1.  Versión básica del word count**

In [0]:
# OPCION 1  se usa si vas a leer desde tu drive el archivo. En este caso usa la libreria que convierte cada entrada del RDD en un objeto tipo Row= (values=('x x  xx]'))
lines=spark.read.text('/gdrive/My Drive/Colab Notebooks/Data/datos.txt').rdd

In [0]:
# OPCION 2usa la libreria basica de SPARk que lee cada entrada del RDD como una cadena de caracteres.
lines1 = sc.textFile("/gdrive/My Drive/Colab Notebooks/Data/datos.txt")

In [0]:
# OPCIONAL  se usa si cargaste el archivo con INST A.
lines=spark.read.text('datos.txt').rdd

In [0]:
# Ejecutar si usó OPCION 1
import re, string
lineLengths = lines.map( lambda s: len(s.value) ) 

In [0]:
#Ejecutar si usó  OPCION 2
lineLengths = lines1.map( lambda s : len(s) ) 

In [0]:
totalLength = lineLengths.reduce( lambda a, b : a + b)

In [0]:
print(totalLength); 

87


In [0]:
lineLengths.take(5)

[15, 12, 27, 12, 21]

# **EXAMPLE 2: Versión del word count eliminando signos de puntuación y otros caracteres**

In [0]:
#  INST A: para subir el archivo a usar en el word count
from google.colab import files
datafile = files.upload()

Saving texto-con-caracteres.txt to texto-con-caracteres.txt


In [0]:
import re, string
text_file = spark.read.text("/gdrive/My Drive/Colab Notebooks/Data/texto-con-caracteres.txt").rdd
punc = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [0]:
# define función a usar en el map
def uni_to_clean_str(x):
    lowercased_str = x.lower()
    lowercased_str = lowercased_str.replace('--',' ')
    clean_str = lowercased_str.translate(punc)
    return clean_str

In [0]:
one_RDD = text_file.flatMap(lambda x: uni_to_clean_str(x.value).split())
one_RDD = one_RDD.map(lambda x: (x,1))
one_RDD = one_RDD.reduceByKey(lambda x,y: x + y)

In [0]:
one_RDD.take(5)

[('pride', 28), ('and', 3370), ('prejudice', 5), ('by', 624), ('jane', 156)]

In [0]:
lines=spark.read.text('/gdrive/My Drive/Colab Notebooks/Data/texto-con-caracteres.txt').rdd

In [0]:
# muesta por qué en el map de debe usar s.value, la estructura de cada entrada del RDD es row(value='xxx')
print(lines.take(2))

[Row(value='PRIDE AND PREJUDICE'), Row(value='')]
